<a href="https://colab.research.google.com/github/kyalan/CUHK-STAT5106-2020/blob/main/Week_8_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initiation of a DataFrame

In [ ]:
import pandas as pd

In [ ]:
data = {
    'apples': [3, 2, 0, 1], 
    'oranges': [0, 3, 7, 2]
}
purchases = pd.DataFrame(data)
print(purchases)

In [ ]:
purchases = pd.DataFrame(data, index=['June', 'Robert', 'Lily', 'David'])
print(purchases)
print(purchases.loc['June'])

# MovieLens

* [Data Source Link](https://grouplens.org/datasets/movielens/)
* The small dataset version is involved – [ml-latest-small.zip](http://files.grouplens.org/datasets/movielens/ml-latest-small.zip), with 100K ratings <br/>
(Of course you can play the larger version if you are interested in)
* 4 files are consisted ([details](http://files.grouplens.org/datasets/movielens/ml-latest-small-README.html))

In [ ]:
# Download file

from urllib.request import urlretrieve
urlretrieve("http://files.grouplens.org/datasets/movielens/ml-latest-small.zip", "ml-latest-small.zip")

In [ ]:
# Directly read csvs in Zip file

import zipfile

zf = zipfile.ZipFile('ml-latest-small.zip') 
df_links = pd.read_csv(zf.open('ml-latest-small/links.csv'), dtype='str')

In [ ]:
print(df_links.info(), "\n")
print(df_links.describe(), "\n")
print(df_links.head())

In [ ]:
df_movies = pd.read_csv(zf.open('ml-latest-small/movies.csv'))
df_ratings = pd.read_csv(zf.open('ml-latest-small/ratings.csv'))
df_tags = pd.read_csv(zf.open('ml-latest-small/tags.csv'))
print(df_movies.shape, df_ratings.shape, df_tags.shape)

In [ ]:
zf.close()

In [ ]:
# unique keys
# guess the key and check it

nrows = df_links.shape[0]
print('df_links no. of rows =', nrows)
# count the number of unique values, and check
print('movieId is the unique keys =', nrows==df_links['movieId'].unique().shape[0])

nrows = df_movies.shape[0]
print('df_movies no. of rows =', nrows)
print('movieId is the unique keys =', nrows==df_movies['movieId'].unique().shape[0])

nrows = df_ratings.shape[0]
print('df_ratings no. of rows =', nrows)
print("['userId', 'movieId', 'timestamp'] are the unique keys ="
      , nrows==df_ratings[['userId', 'movieId', 'timestamp']].drop_duplicates().shape[0])

nrows = df_tags.shape[0]
print('df_tags no. of rows =', nrows)
print("No combinations of columns are the unique keys.")

In [ ]:
#Find the titles of 3 movies, with the most nos. of ratings
df = df_ratings.merge(df_movies, how='left', on='movieId')
print(df.title.value_counts().head(3))

In [ ]:
#Find the no. of movies, with the highest ratings
df_maxRating = df[df.rating==df.rating.max()]
# df_maxRating = df.query('rating==rating.max()')
print(df_maxRating.movieId.unique().shape[0], 'movies get the highest ratings')

In [ ]:
df['datetime'].dt.month

In [ ]:
#Find the peak month users rate movies
df['datetime'] = pd.to_datetime(df.timestamp, unit='s')
df['months'] = df['datetime'].dt.month
print(f'{df["months"].value_counts().index.values[0]}th month is the peak month')

In [ ]:
#Find the peak hour users rate movies
df['hours'] = df['datetime'].dt.hour
print(f'{df["hours"].value_counts().index.values[0]}th hour is the peak hour')

In [ ]:
#Find the movie having the most tags
df = df_tags.merge(df_movies, how='left', on='movieId')
print(df.title.value_counts().head(1))

In [ ]:
#Find the most active userID on tagging movies
print('UserId', df.userId.value_counts().index.values[0], 'is the most active user.')

In [ ]:
#Find the most common tags on those movies
print('Most common tag is', df.tag.value_counts().index.values[0])

# Chunk in read_csv

In [ ]:
zf = zipfile.ZipFile('ml-latest-small.zip') 
iter_df_movies = pd.read_csv(zf.open('ml-latest-small/movies.csv'), chunksize=1000)

In [ ]:
# iterate for each small chunk
df_movies_mini = next(iter_df_movies)
print(df_movies_mini.head(1), df_movies_mini.tail(1))

In [ ]:
# for loops
for df_movies_mini in iter_df_movies:
    print("Reading rows", df_movies_mini.index[0], df_movies_mini.index[-1])

# Generators

In [ ]:
def gen_example():
    n = 1
    print('This is printed first')
    # Generator function contains yield statements
    yield n

    n += 1
    print('This is printed second')
    yield n

    n += 1
    print('This is printed at last')
    yield n 

In [ ]:
g_eg = gen_example()
print(g_eg)

In [ ]:
print(next(g_eg))

In [ ]:
def processing_rows(zipfilename, csvfile):
    zf_gen = zipfile.ZipFile(zipfilename)
    iter_df_movies = pd.read_csv(zf.open(csvfile), chunksize=1000)
    for df_movies_mini in iter_df_movies:
        yield (df_movies_mini.index[0], df_movies_mini.index[-1])
        
g_df = processing_rows('ml-latest-small.zip', 'ml-latest-small/movies.csv')

In [ ]:
for g_df_out in g_df:
    print("Reading rows", g_df_out)